In [79]:
# Import necessary libraries for data handling, preprocessing,
# and building a deep learning model
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers, optimizers
import keras
from sklearn.preprocessing import StandardScaler


In [80]:
# Load the dataset and preview the first few rows

df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [81]:
# Separate features and target variable, then standardize the feature data

X = df.drop('Outcome', axis=1)
y = df['Outcome']

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [82]:
# Build a Sequential neural network model
# Layer 1: Dense layer with 32 neurons, ReLU activation, and input shape matching feature count
# Layer 2: Dense layer with 16 neurons and ReLU activation
# Output Layer: Dense layer with 1 neuron and sigmoid activation for binary classification


model = Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(X.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
# compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4999 - loss: 0.7222
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6526 - loss: 0.6511 
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7062 - loss: 0.6020 
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7799 - loss: 0.5504 
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7913 - loss: 0.5283 
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7646 - loss: 0.5165 
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7683 - loss: 0.4878 
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7768 - loss: 0.4756 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7990 - loss: 0.4465
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7613 - loss: 0.4618 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7974 - loss: 0.4382 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

In [95]:
# check model accuracy
loss, accuracy = model.evaluate(X_train, y_train)
print(f'Accuracy: {accuracy}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8611 - loss: 0.3343
Accuracy: 0.8583061695098877


In [86]:
# lets install keras tuner for parameter tuning
!pip install keras-tuner -q

In [87]:
# Import Keras Tuner for hyperparameter optimization

import keras_tuner

In [113]:
#  Define a tunable neural network model for hyperparameter optimization using Keras Tuner
# - First hidden layer: tunable units (16–128) with ReLU activation
# - Additional hidden layers: tunable count (2–5), units (8–64), dropout rate (0.0–0.5)
# - Output layer: 1 neuron with sigmoid activation for binary classification
# - Optimizer: Adam with tunable learning rate (1e-4 to 1e-1)

def model_tuning(hp):
    model = Sequential()

    # First hidden layer
    model.add(layers.Dense(
        units=hp.Int('units', min_value=16, max_value=128, step=16),
        activation='relu',
        input_shape=(X.shape[1],)
    ))

    # Tunable number of additional hidden layers
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=8, max_value=64, step=8),
            kernel_initializer=keras.initializers.HeNormal(),
            activation='relu'
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_' + str(i), 0.0, 0.5, step=0.1)))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('lr', min_value=1e-4, max_value=1e-1, sampling="log")
        ),

        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [114]:
# Set up Keras Tuner (Random Search) for hyperparameter tuning

tuner = keras_tuner.RandomSearch(
    model_tuning,
    objective='val_accuracy',
    max_trials=10,                  # More attempts
    executions_per_trial=3,         # Average results
    overwrite=True,
    directory='project_5',
    project_name='diabetes_ann'
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [115]:
# Run hyperparameter search
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Trial 10 Complete [00h 01m 49s]
val_accuracy: 0.7683982451756796

Best val_accuracy So Far: 0.7878787716229757
Total elapsed time: 00h 17m 27s


In [116]:
# Display tuning results summary
tuner.results_summary()

Results summary
Results in project_5/diabetes_ann
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
units: 48
num_layers: 2
units_0: 64
dropout_0: 0.1
units_1: 8
dropout_1: 0.30000000000000004
lr: 0.002519122496472792
units_2: 16
dropout_2: 0.30000000000000004
units_3: 64
dropout_3: 0.1
units_4: 24
dropout_4: 0.1
Score: 0.7878787716229757

Trial 03 summary
Hyperparameters:
units: 48
num_layers: 3
units_0: 8
dropout_0: 0.2
units_1: 48
dropout_1: 0.2
lr: 0.0001353565893911894
units_2: 64
dropout_2: 0.0
units_3: 56
dropout_3: 0.30000000000000004
units_4: 8
dropout_4: 0.2
Score: 0.7857142686843872

Trial 00 summary
Hyperparameters:
units: 48
num_layers: 5
units_0: 8
dropout_0: 0.2
units_1: 24
dropout_1: 0.1
lr: 0.021219485642851272
units_2: 8
dropout_2: 0.0
units_3: 8
dropout_3: 0.0
units_4: 8
dropout_4: 0.0
Score: 0.7835497657457987

Trial 02 summary
Hyperparameters:
units: 64
num_layers: 4
units_0: 48
dropout_0: 0.0
units_1: 8
dropo